<a href="https://colab.research.google.com/github/kansalharshit22/first-project/blob/main/hand_tracking_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install mediapipe opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatib

In [2]:
from google.colab import files
uploaded = files.upload()


Saving WIN_20250615_11_26_49_Pro.mp4 to WIN_20250615_11_26_49_Pro.mp4


In [1]:
import mediapipe as mp
import numpy as np
import cv2

# MediaPipe setup
draw_util = mp.solutions.drawing_utils
draw_hand = mp.solutions.hands

# Input video
video_path = 'WIN_20250615_11_26_49_Pro.mp4'
cap = cv2.VideoCapture(video_path)

# Output video setup
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
out = cv2.VideoWriter('output_hand_tracking.mp4', fourcc, 20.0, (640, 480))

# MediaPipe processing
with draw_hand.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (640, 480))
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        image.flags.writeable = False
        results = hands.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image.flags.writeable = True

        # Corrected loop to map handedness to each hand
        if results.multi_hand_landmarks and results.multi_handedness:
            for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                draw_util.draw_landmarks(image, hand_landmarks, draw_hand.HAND_CONNECTIONS,
                                         draw_util.DrawingSpec(color=(122, 22, 76), thickness=2, circle_radius=3),
                                         draw_util.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=3))

                # Extract label and score from handedness classification
                result = hand_handedness.classification[0]
                text = f"{result.label} - {round(result.score, 2)}"
                coords = tuple(np.multiply(
                    np.array((hand_landmarks.landmark[draw_hand.HandLandmark.WRIST].x,
                              hand_landmarks.landmark[draw_hand.HandLandmark.WRIST].y)),
                    [640, 480]).astype(int))

                cv2.putText(image, text, coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        out.write(image)  # Write frame to output video

cap.release()
out.release()
print("✅ Processing complete. Video saved as 'output_hand_tracking.mp4'")


✅ Processing complete. Video saved as 'output_hand_tracking.mp4'
